In [23]:
from pydantic import BaseModel, ValidationError, EmailStr, Field
import json
from typing import List, Literal, Optional
import os
import re
from datetime import datetime
from dotenv import load_dotenv

In [43]:
user_input_json = '''
{
    "name": "John Doe",
    "email": "aftab@example.com",
    "query": "I forgot my Password",
    "order_id": null,
    "purchase_date": null
}
'''

In [44]:
class UserInput(BaseModel):
    name:str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description = "Five digitals are allowed (not start with zero)",
        ge = 10000,
        le = 99999
    )
    purchase_date: Optional[datetime]

In [45]:
user_input = UserInput.model_validate_json(user_input_json)

In [46]:
# Define the CustomerQuery model that inherits from UserInput
class CustomerQuery(UserInput):
    priority: str = Field(
        ..., description="Priority level: low, medium, high"
    )
    category: Literal[
        'refund_request', 'information_request', 'other'
    ] = Field(..., description="Query category")
    is_complaint: bool = Field(
        ..., description="Whether this is a complaint"
    )
    tags: List[str] = Field(..., description="Relevant keyword tags")

In [47]:
# Create a prompt with generic example data to guide LLM.
example_response_structure = f"""{{
    name="Example User",
    email="user@example.com",
    query="I ordered a new computer monitor and it arrived with the screen cracked. I need to exchange it for a new one.",
    order_id=12345,
    purchase_date="2025-12-31",
    priority="medium",
    category="refund_request",
    is_complaint=True,
    tags=["monitor", "support", "exchange"] 
}}"""


In [48]:
# Create prompt with user data and expected JSON structure
prompt = f"""
Please analyze this user query\n {user_input.model_dump_json(indent=2)}:

Return your analysis as a JSON object matching this exact structure 
and data types:
{example_response_structure}

Respond ONLY with valid JSON. Do not include any explanations or 
other text or formatting before or after the JSON object.
"""

print(prompt)


Please analyze this user query
 {
  "name": "John Doe",
  "email": "aftab@example.com",
  "query": "I forgot my Password",
  "order_id": null,
  "purchase_date": null
}:

Return your analysis as a JSON object matching this exact structure 
and data types:
{
    name="Example User",
    email="user@example.com",
    query="I ordered a new computer monitor and it arrived with the screen cracked. I need to exchange it for a new one.",
    order_id=12345,
    purchase_date="2025-12-31",
    priority="medium",
    category="refund_request",
    is_complaint=True,
    tags=["monitor", "support", "exchange"] 
}

Respond ONLY with valid JSON. Do not include any explanations or 
other text or formatting before or after the JSON object.

